# Ray Tracing from scratch

In [ ]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt
import math


In [ ]:
#function for normalisation
def normalize(tensor): 
  norm= tensor/torch.norm(tensor)

  return norm 

# if the ray (line) that starts at camera and goes towards p intersects any object of the scene then...
#Calculate intersection point 
def object_intersection(OB_centre, radius, ray_direction,ray_origin ): 
  a= 1
  sub= ray_origin- OB_centre
  sub = sub.type(torch.float)

  b= 2*torch.dot(ray_direction,sub)
  c=torch.norm(ray_origin-OB_centre)**2 - radius**2

  det=b**2- 4* c

  if det> 0: 
    t1= (-b + torch.sqrt(det))/2
    t2= (-b - torch.sqrt(det))/2

    if t1 > 0 and t2>0:  # only when t1 and t2 are positive 
    # negative t1 and t2 values means the intersection is behine the camera and screen
      return min(t1,t2) # only return the nearest intersection
  return None

# calculating the intersection point to the nearest object
# to check if it is shadowed or not
def nearest_object(objects, ray_origin, ray_direction):
  distances= [object_intersection(obj['center'], obj['radius'],ray_direction, ray_origin ) for obj in objects]

  nearest_obj= None
  min_distance= math.inf

  for idx, dist in enumerate(distances): 
    if dist and dist< min_distance: 
      min_distance= dist
      nearest_obj = objects[idx]

  return nearest_obj, min_distance




In [ ]:

# STEP 1: SETTING UP THE SCENE
height= 200
width= 300

camera_eye= torch.tensor([0., 0., 1.])

ratio= width/height
screen= (-1., 1/ratio, 1., -1/ratio)
#defining the spheres 
objects = [
    { 'center': torch.tensor([-0.2, 0, -1]), 'radius': 0.7, 'ambient': torch.tensor([0.1, 0., 0.]), 
     'diffuse': torch.tensor([0.7, 0., 0.]), 'specular': torch.tensor([1., 1., 1.]), 'shininess': 100},

    { 'center': torch.tensor([0.1, -0.3, 0.]), 'radius': 0.1, 'ambient': torch.tensor([0.1, 0., 0.1]),
     'diffuse': torch.tensor([0.7, 0., 0.7]), 'specular': torch.tensor([1., 1., 1.]), 'shininess': 100 },

    { 'center': torch.tensor([-0.3, 0., 0.]), 'radius': 0.15, 'ambient': torch.tensor([0., 0.1, 0.]),
     'diffuse': torch.tensor([0., 0.6, 0.]), 'specular': torch.tensor([1., 1., 1.]), 'shininess': 100 },

     { 'center': torch.tensor([0, -9000, 0]), 'radius': 9000 - 0.7, 'ambient': torch.tensor([0.1, 0.1, 0.1]), 
      'diffuse': torch.tensor([0.6, 0.6, 0.6]), 'specular': torch.tensor([1., 1., 1.]), 'shininess': 100 }
]

#Light definition 
light= {'position': torch.tensor([5.,5.,5.]), 'ambient': torch.tensor([1., 1., 1.]), 
        'diffuse': torch.tensor([1., 1., 1.]), 'specular': torch.tensor([1., 1., 1.])}

image=torch.zeros((height, width, 3))

for i, y in enumerate(torch.linspace(screen[1], screen[3], height)):
  for j, x in enumerate(torch.linspace(screen[0], screen[2], width)):
    print("progress: %d/%d" % (i + 1, height))
    #Creating a ray from the ray equation 
    # ray= camera+ [(pixel- camera)/norm((pixel- camera))] * timetep(destination form camera)

    origin=camera_eye
    pixel= torch.tensor([x, y, 0])
    direction= normalize(pixel-origin)

    nearest_obj, dist_from_O= nearest_object(objects, origin, direction)
   

    if nearest_obj ==None: 
      continue
    intersection_point= origin + direction * dist_from_O


    #finding the normal to the sphere
    norm_to_surface = normalize(intersection_point - nearest_obj['center'])
    new_origin = intersection_point + 1e-5*norm_to_surface    
    intersection_to_lightR =normalize(light['position'] - new_origin)

    

    _, distance_from_inter = nearest_object(objects,new_origin, intersection_to_lightR)

    intersection_to_lightD = torch.norm(light['position'] - intersection_point )
    isit_shadow= distance_from_inter< intersection_to_lightD
    #check if it is shadowed
    if isit_shadow:
      continue


   # Additing illumination to the object and determining the colour of each of the sphere by using tthe phong model

    illumination = torch.zeros((3))

    # Ambient
    illumination += nearest_obj['ambient'] * light['ambient']

    # diffuse
    illumination += nearest_obj['diffuse'] * light['diffuse'] * torch.dot(intersection_to_lightR,norm_to_surface)

   # specular
    intersection_to_camera = normalize(camera_eye - intersection_point)
    shy=nearest_obj['shininess']/4
    norm = normalize(intersection_to_lightR + intersection_to_camera)

    illumination += nearest_obj['specular'] * light['specular'] * torch.dot(norm_to_surface, norm )**shy


    # final image

    image[i, j] =torch.clip(illumination, 0, 1)    


plt.figure()
plt.title('The Scene of the simulation')
plt.imshow(image)

